In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
spec = ADatabase("forex")
umod = UniversalModeler()

In [3]:
factors = [str(x) for x in range(14)]

In [4]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [5]:
included_columns.extend(factors)

In [6]:
training_sets = []

In [7]:
market.connect()
data = market.retrieve("forex")
market.disconnect()

In [10]:
all_prices = data.melt(id_vars="date",value_vars=[x for x in data.columns if x != "date"]).rename(columns={"variable":"ticker","value":"adjclose"})

In [11]:
all_prices

,date,ticker,adjclose
0,2000-01-01,ANG,1.790000
1,2000-01-02,ANG,1.790000
2,2000-01-03,ANG,1.790000
3,2000-01-04,ANG,1.790000
4,2000-01-05,ANG,1.790000
...,...,...,...
1446015,2023-04-11,VES,24.361623
1446016,2023-04-12,VES,24.442814
1446017,2023-04-13,VES,24.457308
1446018,2023-04-14,VES,24.511235


In [12]:
for ticker in tqdm(all_prices["ticker"].unique()):
    try:
        prices = all_prices[all_prices["ticker"]==ticker]
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        ticker_data = prices[prices["ticker"]==ticker]
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
        for i in range(14):
            ticker_data[str(i)] = ticker_data["adjclose"].shift(i)
        ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
        ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
        ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
        ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data["y"] = ticker_data["adjclose"].shift(-1)
        ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
        ticker_data.dropna(inplace=True)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 170/170 [00:19<00:00,  8.56it/s]


In [13]:
data = pd.concat(training_sets)

In [14]:
data = data.dropna()

In [15]:
spec.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2009,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors)
        stuff["training_years"] = years
        spec.store(set_name,stuff)
spec.disconnect()


  0%|                                                                                                                     | 0/14 [00:00<?, ?it/s]

Epoch 1/10
636/636 [==============================] - 1s 837us/step - loss: 138776.4219 - mean_absolute_percentage_error: 7.3704
Epoch 2/10
636/636 [==============================] - 0s 753us/step - loss: 15177.1963 - mean_absolute_percentage_error: 6.2005
Epoch 3/10
636/636 [==============================] - 0s 750us/step - loss: 14028.9170 - mean_absolute_percentage_error: 5.5299
Epoch 4/10
636/636 [==============================] - 0s 748us/step - loss: 8247.0654 - mean_absolute_percentage_error: 5.9860
Epoch 5/10
636/636 [==============================] - 0s 756us/step - loss: 5624.3604 - mean_absolute_percentage_error: 6.2836
Epoch 6/10
636/636 [==============================] - 0s 752us/step - loss: 7862.6318 - mean_absolute_percentage_error: 7.0520
Epoch 7/10
636/636 [==============================] - 0s 747us/step - loss: 8175.4668 - mean_absolute_percentage_error: 4.3493
Epoch 8/10
636/636 [==============================] - 0s 750us/step - loss: 9193.1445 - mean_absolute_perce

  7%|███████▊                                                                                                     | 1/14 [00:27<05:55, 27.38s/it]

Epoch 1/10
647/647 [==============================] - 1s 796us/step - loss: 54573.9102 - mean_absolute_percentage_error: 4.6424
Epoch 2/10
647/647 [==============================] - 0s 771us/step - loss: 10275.9229 - mean_absolute_percentage_error: 3.8243
Epoch 3/10
647/647 [==============================] - 0s 771us/step - loss: 8014.5986 - mean_absolute_percentage_error: 2.6303
Epoch 4/10
647/647 [==============================] - 1s 779us/step - loss: 6617.9116 - mean_absolute_percentage_error: 3.4762
Epoch 5/10
647/647 [==============================] - 0s 765us/step - loss: 17375.6348 - mean_absolute_percentage_error: 4.6031
Epoch 6/10
647/647 [==============================] - 1s 782us/step - loss: 5121.3291 - mean_absolute_percentage_error: 5.9867
Epoch 7/10
647/647 [==============================] - 0s 766us/step - loss: 9091.6709 - mean_absolute_percentage_error: 3.8941
Epoch 8/10
647/647 [==============================] - 0s 757us/step - loss: 4943.3267 - mean_absolute_percen

 14%|███████████████▌                                                                                             | 2/14 [00:54<05:27, 27.29s/it]

...

 93%|████████████████████████████████████████████████████████████████████████████████████████████████████▎       | 13/14 [06:59<00:38, 38.93s/it]

Epoch 1/10
768/768 [==============================] - 1s 947us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 2/10
768/768 [==============================] - 1s 950us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 3/10
768/768 [==============================] - 1s 944us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 4/10
768/768 [==============================] - 1s 950us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 5/10
768/768 [==============================] - 1s 926us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 6/10
768/768 [==============================] - 1s 934us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 7/10
768/768 [==============================] - 1s 951us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 8/10
768/768 [==============================] - 1s 931us/step - loss: nan - mean_absolute_percentage_error: nan
Epoch 9/10
768/768 [==============================] - 1s

 93%|████████████████████████████████████████████████████████████████████████████████████████████████████▎       | 13/14 [07:46<00:35, 35.90s/it]

'str' object has no attribute 'predict'


KeyError: 'tf_prediction'